In [1]:
import os, json, re
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterstats import zonal_stats
import osmnx as ox
import statsmodels.api as sm
from shapely.geometry import box, Point
from shapely.geometry import mapping
from IPython.display import IFrame

# -----------------------------
# Project / output folders
# -----------------------------
DATA_DIR = "data/yerevan_full_pipeline"
os.makedirs(DATA_DIR, exist_ok=True)

# -----------------------------
# Grid settings
# -----------------------------
GRID_CELL_M = 500          # 250 gives more detail, larger HTML
BUFFER_HERIT_M = 500       # heritage count radius

# -----------------------------
# Transport proxy settings
# -----------------------------
AVG_SPEED_KMH = 20.0
SPEED_M_PER_MIN = AVG_SPEED_KMH * 1000.0 / 60.0

# -----------------------------
# Rent conversion settings
# -----------------------------
GROSS_YIELD_ANNUAL = 0.06  # implied rent from sale price per m2
FILTER_CURRENCY = "USD"    # keep USD only for Kaggle sale listing

# -----------------------------
# Inputs
# -----------------------------
WORLDPOP_URL = (
    "https://data.worldpop.org/GIS/Population/Individual_countries/ARM/"
    "Armenia_100m_Population/ARM_ppp_v2c_2020.tif"
)

KAGGLE_SALE_CSV = r"data/kaggle_real_estate/apartments_for_sale(with_lat_long).csv"

# Historic center g (Republic Square) fixed coordinates
G_LON, G_LAT = 44.5126, 40.1775  # lon, lat


In [2]:
def ensure_crs(gdf, epsg=32638):
    if gdf.crs is None:
        raise RuntimeError("GeoDataFrame CRS missing.")
    return gdf.to_crs(epsg=epsg)

def download_file(url, out_path):
    import requests
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
        return out_path
    r = requests.get(url, stream=True, timeout=180)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)
    return out_path

def get_yerevan_boundary():
    ox.settings.use_cache = True
    ox.settings.log_console = True
    ox.settings.timeout = 180

    gdf = ox.geocode_to_gdf("Yerevan, Armenia")
    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
    if gdf.empty:
        raise RuntimeError("Could not fetch Yerevan polygon from OSM.")
    try:
        geom = gdf.geometry.union_all()
    except Exception:
        geom = gdf.geometry.unary_union
    out = gpd.GeoDataFrame({"name":["Yerevan"]}, geometry=[geom], crs=gdf.crs)
    return out

def clip_raster_to_polygon(in_tif, poly_gdf, out_tif):
    with rasterio.open(in_tif) as src:
        poly = poly_gdf.to_crs(src.crs)
        geom = poly.geometry.iloc[0]
        geoms = [mapping(geom)]
        out_img, out_transform = mask(src, geoms, crop=True, filled=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform
        })
        os.makedirs(os.path.dirname(out_tif), exist_ok=True)
        with rasterio.open(out_tif, "w", **out_meta) as dst:
            dst.write(out_img)
    return out_tif

def reproject_raster(in_tif, out_tif, dst_crs="EPSG:32638"):
    os.makedirs(os.path.dirname(out_tif), exist_ok=True)
    with rasterio.open(in_tif) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({"crs": dst_crs, "transform": transform, "width": width, "height": height})
        with rasterio.open(out_tif, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
    return out_tif

def make_grid(bounds, cell_size_m, crs):
    minx, miny, maxx, maxy = bounds
    xs = np.arange(minx, maxx + cell_size_m, cell_size_m)
    ys = np.arange(miny, maxy + cell_size_m, cell_size_m)
    cells = []
    for x in xs[:-1]:
        for y in ys[:-1]:
            cells.append(box(x, y, x + cell_size_m, y + cell_size_m))
    grid = gpd.GeoDataFrame({"geometry": cells}, crs=crs)
    grid["cell_id"] = np.arange(len(grid)).astype(int)
    return grid

def parse_area(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace(",", ".")
    m = re.search(r"(\d+(\.\d+)?)", s)
    return float(m.group(1)) if m else np.nan

def utm_xy_to_lonlat(x, y):
    pt = gpd.GeoSeries([Point(x, y)], crs="EPSG:32638").to_crs("EPSG:4326").iloc[0]
    return float(pt.x), float(pt.y)


In [3]:
yerevan_ll = get_yerevan_boundary()               # EPSG:4326 usually
yerevan_utm = ensure_crs(yerevan_ll, 32638)

BOUNDARY_GPKG = os.path.join(DATA_DIR, "yerevan_boundary.gpkg")
yerevan_utm.to_file(BOUNDARY_GPKG, layer="boundary", driver="GPKG")

print("Saved boundary:", BOUNDARY_GPKG)


Saved boundary: data/yerevan_full_pipeline\yerevan_boundary.gpkg


In [4]:
raw_tif = os.path.join(DATA_DIR, "worldpop_ARM_2020.tif")
clip_tif = os.path.join(DATA_DIR, "worldpop_yerevan_clipped_wgs84.tif")
clip_utm_tif = os.path.join(DATA_DIR, "worldpop_yerevan_clipped_utm38n.tif")

download_file(WORLDPOP_URL, raw_tif)
clip_raster_to_polygon(raw_tif, yerevan_ll, clip_tif)
reproject_raster(clip_tif, clip_utm_tif, dst_crs="EPSG:32638")

print("Saved raster:", clip_utm_tif)


Saved raster: data/yerevan_full_pipeline\worldpop_yerevan_clipped_utm38n.tif


In [5]:
grid = make_grid(yerevan_utm.total_bounds, GRID_CELL_M, crs=yerevan_utm.crs)
grid = gpd.overlay(grid, yerevan_utm[["geometry"]], how="intersection")
grid["area_km2"] = grid.geometry.area / 1_000_000.0
grid["cx"] = grid.geometry.centroid.x
grid["cy"] = grid.geometry.centroid.y

GRID_GPKG = os.path.join(DATA_DIR, f"yerevan_grid_{GRID_CELL_M}m.gpkg")
grid.to_file(GRID_GPKG, layer="grid", driver="GPKG")

print("Saved grid:", GRID_GPKG, "cells:", len(grid))


Saved grid: data/yerevan_full_pipeline\yerevan_grid_500m.gpkg cells: 1036


In [6]:
grid = make_grid(yerevan_utm.total_bounds, GRID_CELL_M, crs=yerevan_utm.crs)
grid = gpd.overlay(grid, yerevan_utm[["geometry"]], how="intersection")
grid["area_km2"] = grid.geometry.area / 1_000_000.0
grid["cx"] = grid.geometry.centroid.x
grid["cy"] = grid.geometry.centroid.y

GRID_GPKG = os.path.join(DATA_DIR, f"yerevan_grid_{GRID_CELL_M}m.gpkg")
grid.to_file(GRID_GPKG, layer="grid", driver="GPKG")

print("Saved grid:", GRID_GPKG, "cells:", len(grid))


Saved grid: data/yerevan_full_pipeline\yerevan_grid_500m.gpkg cells: 1036


In [7]:
with rasterio.open(clip_utm_tif) as src:
    nodata = src.nodata

zs = zonal_stats(grid, clip_utm_tif, stats=["sum"], nodata=nodata, all_touched=False)
grid_pop = grid.copy()
grid_pop["pop_sum"] = [z["sum"] if z["sum"] is not None else 0.0 for z in zs]
grid_pop["pop_density_per_km2"] = grid_pop["pop_sum"] / grid_pop["area_km2"].replace(0, np.nan)

POP_GPKG = os.path.join(DATA_DIR, "grid_population.gpkg")
grid_pop.to_file(POP_GPKG, layer="grid_pop", driver="GPKG")
print("Saved population grid:", POP_GPKG)


Saved population grid: data/yerevan_full_pipeline\grid_population.gpkg


In [8]:
ox.settings.use_cache = True
ox.settings.log_console = True
ox.settings.timeout = 180

BUSINESS_TAGS = {
    "shop": True,
    "office": True,
    "craft": True,
    "industrial": True,
    "amenity": ["restaurant","cafe","bar","fast_food","bank","pharmacy","hospital","clinic"],
    "building": ["commercial","industrial","retail","office"]
}

poly_ll = yerevan_ll.geometry.iloc[0]
biz_raw = ox.features_from_polygon(poly_ll, BUSINESS_TAGS)
biz_raw = biz_raw[biz_raw.geometry.notnull()].copy()

biz = ensure_crs(biz_raw, 32638)
gt = biz.geometry.geom_type
pts = biz[gt == "Point"].copy()
polys = biz[gt.isin(["Polygon","MultiPolygon"])].copy()
polys["geometry"] = polys.geometry.representative_point()
biz_pts = gpd.GeoDataFrame(pd.concat([pts, polys], ignore_index=True), crs="EPSG:32638")

# Spatial join to grid
join = gpd.sjoin(biz_pts[["geometry"]], grid[["cell_id","geometry"]], predicate="within", how="left")
counts = join.groupby("cell_id").size().rename("biz_count").reset_index()

grid_biz = grid.copy()
grid_biz = grid_biz.merge(counts, on="cell_id", how="left")
grid_biz["biz_count"] = grid_biz["biz_count"].fillna(0).astype(int)
grid_biz["biz_density_per_km2"] = grid_biz["biz_count"] / grid_biz["area_km2"].replace(0, np.nan)

# Keep only safe columns for saving points (you don't need all OSM attributes)
safe_cols = [c for c in ["name", "shop", "office", "craft", "industrial", "amenity", "building"] if c in biz_pts.columns]
biz_pts_out = biz_pts[safe_cols + ["geometry"]].copy()

BIZ_POINTS_GPKG = os.path.join(DATA_DIR, "business_points.gpkg")
BIZ_GRID_GPKG = os.path.join(DATA_DIR, "grid_business.gpkg")

biz_pts_out.to_file(BIZ_POINTS_GPKG, layer="biz_points", driver="GPKG")
grid_biz.to_file(BIZ_GRID_GPKG, layer="grid_biz", driver="GPKG")

print("Saved business points:", BIZ_POINTS_GPKG, "n:", len(biz_pts))
print("Saved business grid:", BIZ_GRID_GPKG)


Saved business points: data/yerevan_full_pipeline\business_points.gpkg n: 10817
Saved business grid: data/yerevan_full_pipeline\grid_business.gpkg


In [9]:
df = pd.read_csv(KAGGLE_SALE_CSV)
for c in ["Latitude","Longitude","price","currency","floor_area"]:
    if c not in df.columns:
        raise RuntimeError(f"Missing column {c} in Kaggle CSV. Columns: {list(df.columns)}")

work = df.dropna(subset=["Latitude","Longitude","price"]).copy()
work["price"] = pd.to_numeric(work["price"], errors="coerce")
work = work.dropna(subset=["price"])

work["currency"] = work["currency"].astype(str).str.upper().str.strip()
work = work[work["currency"] == FILTER_CURRENCY].copy()

work["area_m2"] = work["floor_area"].apply(parse_area)
work.loc[(work["area_m2"] < 10) | (work["area_m2"] > 500), "area_m2"] = np.nan
work["price_per_m2"] = work["price"] / work["area_m2"]
work.loc[(work["price_per_m2"] <= 0) | (work["price_per_m2"] > 20000), "price_per_m2"] = np.nan

# Implied rent per m2 per month
work["implied_rent_per_m2_month"] = (work["price_per_m2"] * GROSS_YIELD_ANNUAL) / 12.0

rent_pts = gpd.GeoDataFrame(
    work,
    geometry=gpd.points_from_xy(work["Longitude"], work["Latitude"]),
    crs="EPSG:4326"
)
# clip to Yerevan
rent_pts = rent_pts[rent_pts.within(yerevan_ll.geometry.iloc[0])].copy()
rent_pts = ensure_crs(rent_pts, 32638)

# Join to grid and aggregate
j = gpd.sjoin(rent_pts[["implied_rent_per_m2_month","geometry"]],
              grid[["cell_id","geometry"]],
              predicate="within", how="left")

agg = j.groupby("cell_id").agg(
    rent_listings=("geometry","size"),
    implied_rent_per_m2_month=("implied_rent_per_m2_month","median")
).reset_index()

grid_rent = grid.copy()
grid_rent = grid_rent.merge(agg, on="cell_id", how="left")
grid_rent["rent_listings"] = grid_rent["rent_listings"].fillna(0).astype(int)

RENT_POINTS_GPKG = os.path.join(DATA_DIR, "rent_points_kaggle_sale.gpkg")
RENT_GRID_GPKG = os.path.join(DATA_DIR, "grid_rent.gpkg")
rent_pts.to_file(RENT_POINTS_GPKG, layer="rent_points", driver="GPKG")
grid_rent.to_file(RENT_GRID_GPKG, layer="grid_rent", driver="GPKG")

print("Saved rent points:", RENT_POINTS_GPKG, "n:", len(rent_pts))
print("Saved rent grid:", RENT_GRID_GPKG)


Saved rent points: data/yerevan_full_pipeline\rent_points_kaggle_sale.gpkg n: 56021
Saved rent grid: data/yerevan_full_pipeline\grid_rent.gpkg


In [10]:
TAGS_HERIT = {
    "historic": True,
    "heritage": True,
    "tourism": ["museum", "attraction"],
    "memorial": True
}
herit_raw = ox.features_from_polygon(yerevan_ll.geometry.iloc[0], TAGS_HERIT)
herit_raw = herit_raw[herit_raw.geometry.notnull()].copy()
herit = ensure_crs(herit_raw, 32638)

gt = herit.geometry.geom_type
hp = herit[gt == "Point"].copy()
hpoly = herit[gt.isin(["Polygon","MultiPolygon"])].copy()
hpoly["geometry"] = hpoly.geometry.representative_point()
herit_pts = gpd.GeoDataFrame(pd.concat([hp, hpoly], ignore_index=True), crs="EPSG:32638")

# g point in UTM
g_utm = gpd.GeoSeries([Point(G_LON, G_LAT)], crs="EPSG:4326").to_crs("EPSG:32638").iloc[0]

amen = grid.copy()
amen_cent = gpd.GeoDataFrame(amen[["cell_id"]].copy(), geometry=amen.geometry.centroid, crs="EPSG:32638")

# distance to g
amen["dist_to_g_m"] = amen.geometry.centroid.distance(g_utm)

# distance to nearest heritage point
nearest = gpd.sjoin_nearest(
    amen_cent,
    herit_pts[["geometry"]],
    how="left",
    distance_col="dist_to_nearest_herit_m"
)
amen = amen.merge(nearest[["cell_id","dist_to_nearest_herit_m"]], on="cell_id", how="left")

# count within buffer
buff = amen_cent.copy()
buff["geometry"] = buff.geometry.buffer(BUFFER_HERIT_M)
join_cnt = gpd.sjoin(herit_pts[["geometry"]], buff[["cell_id","geometry"]], predicate="within", how="left")
cnt = join_cnt.groupby("cell_id").size().rename("herit_cnt_500m").reset_index()
amen = amen.merge(cnt, on="cell_id", how="left")
amen["herit_cnt_500m"] = amen["herit_cnt_500m"].fillna(0).astype(int)

buffer_area_km2 = (np.pi * (BUFFER_HERIT_M ** 2)) / 1_000_000.0
amen["herit_density_500m"] = amen["herit_cnt_500m"] / buffer_area_km2

AMEN_POINTS_GPKG = os.path.join(DATA_DIR, "heritage_points.gpkg")
AMEN_GRID_GPKG = os.path.join(DATA_DIR, "grid_amenity.gpkg")
herit_pts.to_file(AMEN_POINTS_GPKG, layer="herit_points", driver="GPKG")
amen.to_file(AMEN_GRID_GPKG, layer="grid_amen", driver="GPKG")

print("Saved heritage points:", AMEN_POINTS_GPKG, "n:", len(herit_pts))
print("Saved amenity grid:", AMEN_GRID_GPKG)


Saved heritage points: data/yerevan_full_pipeline\heritage_points.gpkg n: 741
Saved amenity grid: data/yerevan_full_pipeline\grid_amenity.gpkg


In [11]:
# Start from canonical grid
master = grid.copy()

# Merge population
gp = ensure_crs(gpd.read_file(POP_GPKG, layer="grid_pop"), 32638)
master = master.merge(gp[["cell_id","pop_sum","pop_density_per_km2"]], on="cell_id", how="left")

# Merge business
gb = ensure_crs(gpd.read_file(BIZ_GRID_GPKG, layer="grid_biz"), 32638)
master = master.merge(gb[["cell_id","biz_count","biz_density_per_km2"]], on="cell_id", how="left")

# Merge rent
gr = ensure_crs(gpd.read_file(RENT_GRID_GPKG, layer="grid_rent"), 32638)
master = master.merge(gr[["cell_id","rent_listings","implied_rent_per_m2_month"]], on="cell_id", how="left")

# Merge amenity
ga = ensure_crs(gpd.read_file(AMEN_GRID_GPKG, layer="grid_amen"), 32638)
master = master.merge(ga[["cell_id","dist_to_g_m","dist_to_nearest_herit_m","herit_cnt_500m","herit_density_500m"]], on="cell_id", how="left")

# Build final variables
master["cx"] = master.geometry.centroid.x
master["cy"] = master.geometry.centroid.y

master["R_implied"] = pd.to_numeric(master["implied_rent_per_m2_month"], errors="coerce")
master["rent_missing"] = (~np.isfinite(master["R_implied"]) | (master["R_implied"] <= 0)).astype(int)
R_med = np.nanmedian(master.loc[master["rent_missing"] == 0, "R_implied"])
master.loc[master["rent_missing"] == 1, "R_implied"] = R_med
master["log_rent"] = np.log1p(master["R_implied"])

# Use observed M(x)
master["M_obs"] = master["biz_density_per_km2"].fillna(0)

# Define y_business as top 30% business intensity
thr = float(2)
master["y_business"] = (master["M_obs"] >= thr).astype(int)

MASTER_GPKG = os.path.join(DATA_DIR, "master_grid.gpkg")
master.to_file(MASTER_GPKG, layer="master", driver="GPKG")
print("Saved master grid:", MASTER_GPKG)
print("Cells:", len(master), "Business share:", master["y_business"].mean())


Saved master grid: data/yerevan_full_pipeline\master_grid.gpkg
Cells: 1036 Business share: 0.4691119691119691


In [12]:
pd.DataFrame(master).to_csv('master.csv')

In [13]:
master

,cell_id,geometry,area_km2,cx,cy,pop_sum,pop_density_per_km2,biz_count,biz_density_per_km2,rent_listings,implied_rent_per_m2_month,dist_to_g_m,dist_to_nearest_herit_m,herit_cnt_500m,herit_density_500m,R_implied,rent_missing,log_rent,M_obs,y_business
0,17,"POLYGON ((446166.158 4444171.848, 446166.158 4...",0.006514,446124.989309,4.444134e+06,2.814809,432.114292,0,0.0,0,NaN,12847.170533,2893.470419,0,0.0,6.588235,1,2.026599,0.0,0
1,18,"POLYGON ((446166.158 4444171.848, 446072.839 4...",0.060923,445978.826383,4.444288e+06,11.043721,181.274731,0,0.0,0,NaN,12948.067531,2998.921624,0,0.0,6.588235,1,2.026599,0.0,0
2,57,"POLYGON ((446166.158 4444171.848, 446666.158 4...",0.039380,446415.982016,4.444132e+06,19.865776,504.460248,0,0.0,0,NaN,12567.421897,2614.658309,0,0.0,6.588235,1,2.026599,0.0,0
3,58,"POLYGON ((446666.158 4444171.848, 446166.158 4...",0.130023,446416.608316,4.444302e+06,76.684990,589.782013,0,0.0,0,NaN,12521.511533,2568.914426,0,0.0,6.588235,1,2.026599,0.0,0
4,97,"POLYGON ((446666.158 4444171.848, 447166.158 4...",0.040908,446922.502942,4.444131e+06,21.588802,527.743532,0,0.0,0,NaN,12081.520311,2136.821358,0,0.0,6.588235,1,2.026599,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1735,"POLYGON ((467166.158 4443171.848, 467452.13 44...",0.023871,467245.151731,4.443090e+06,1.349567,56.535276,0,0.0,0,NaN,9823.892902,5509.395672,0,0.0,6.588235,1,2.026599,0.0,0
1032,1736,"POLYGON ((467166.158 4443171.848, 467166.158 4...",0.212524,467384.653723,4.443445e+06,16.280460,76.605206,0,0.0,0,NaN,9793.112529,5331.017994,0,0.0,6.588235,1,2.026599,0.0,0
1033,1737,"POLYGON ((467666.158 4443671.848, 467166.158 4...",0.054762,467389.670477,4.443729e+06,3.182314,58.112105,0,0.0,0,NaN,9681.595876,5129.171829,0,0.0,6.588235,1,2.026599,0.0,0
1034,1776,"POLYGON ((467666.158 4443671.848, 467757.713 4...",0.006858,467696.675923,4.443622e+06,0.000000,0.000000,0,0.0,0,NaN,10005.923454,5423.226859,0,0.0,6.588235,1,2.026599,0.0,0


In [14]:
# Initial mu from observed business intensity
w = master["M_obs"].fillna(0).to_numpy()
mu0x = float((master["cx"].to_numpy() * w).sum() / w.sum())
mu0y = float((master["cy"].to_numpy() * w).sum() / w.sum())

def tau_minutes(cx, cy, mux, muy, transport_factor=1.0):
    d = np.sqrt((cx - mux)**2 + (cy - muy)**2)
    return transport_factor * (d / SPEED_M_PER_MIN)

# Baseline tau for estimation
master["tau_min"] = tau_minutes(master["cx"].to_numpy(), master["cy"].to_numpy(), mu0x, mu0y, 1.0)

features = [
    "tau_min",
    "dist_to_g_m",
    "herit_density_500m",
    "pop_density_per_km2",
    "log_rent",
    "rent_missing"
]

est = master[["y_business"] + features].replace([np.inf,-np.inf], np.nan).dropna().copy()
means = est[features].mean()
stds = est[features].std().replace(0, 1.0)

def zscore(dfX):
    Z = dfX.copy()
    for c in dfX.columns:
        Z[c] = (Z[c] - means[c]) / stds[c]
    return Z

X = sm.add_constant(zscore(est[features]), has_constant="add")
y = est["y_business"].astype(int)
logit_model = sm.Logit(y, X).fit(disp=False)
print(logit_model.summary())

def predict_shares(df, mux, muy, transport_factor, amenity_factor):
    tmp = df.copy()
    tmp["tau_min"] = tau_minutes(tmp["cx"].to_numpy(), tmp["cy"].to_numpy(), mux, muy, transport_factor)

    Xraw = tmp[features].replace([np.inf,-np.inf], np.nan)
    Z = zscore(Xraw)

    # Amenity slider scales historic components in standardized space
    Z["dist_to_g_m"] = Z["dist_to_g_m"] * amenity_factor
    Z["herit_density_500m"] = Z["herit_density_500m"] * amenity_factor

    Xmat = sm.add_constant(Z, has_constant="add")
    p = logit_model.predict(Xmat)
    return p.to_numpy()

def solve_mu(df, transport_factor, amenity_factor, mu_init=(mu0x, mu0y), max_iters=30, tol_m=30.0):
    mux, muy = mu_init
    cx = df["cx"].to_numpy()
    cy = df["cy"].to_numpy()

    for _ in range(max_iters):
        s = predict_shares(df, mux, muy, transport_factor, amenity_factor)
        valid = np.isfinite(s)
        s2 = s[valid]
        if s2.sum() <= 1e-9:
            break
        mux_new = float((cx[valid] * s2).sum() / s2.sum())
        muy_new = float((cy[valid] * s2).sum() / s2.sum())
        shift = float(np.sqrt((mux_new - mux)**2 + (muy_new - muy)**2))
        mux, muy = mux_new, muy_new
        if shift < tol_m:
            break

    s_final = predict_shares(df, mux, muy, transport_factor, amenity_factor)
    return (mux, muy), s_final


                           Logit Regression Results                           
Dep. Variable:             y_business   No. Observations:                 1036
Model:                          Logit   Df Residuals:                     1029
Method:                           MLE   Df Model:                            6
Date:                Wed, 04 Feb 2026   Pseudo R-squ.:                  0.4279
Time:                        10:31:18   Log-Likelihood:                -409.71
converged:                       True   LL-Null:                       -716.12
Covariance Type:            nonrobust   LLR p-value:                4.005e-129
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.2523      0.145      1.739      0.082      -0.032       0.537
tau_min                -0.7644      0.754     -1.013      0.311      -2.243       0.714
dist_to_g_m     

In [15]:
import os, json
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

OUT_DIR = "data/yerevan_interactive"
os.makedirs(OUT_DIR, exist_ok=True)

# Use a simplified GeoJSON so the HTML stays light
geo = master[["cell_id", "geometry"]].copy().reset_index(drop=True)
geo["gid"] = np.arange(len(geo)).astype(int)
geo_ll = geo[["gid", "geometry"]].to_crs("EPSG:4326")
geo_ll["geometry"] = geo_ll["geometry"].simplify(0.00005, preserve_topology=True)
geojson_str = geo_ll.to_json()

# Extract arrays (one value per gid)
cx = master["cx"].to_numpy().astype(float)
cy = master["cy"].to_numpy().astype(float)
dist_g = master["dist_to_g_m"].to_numpy().astype(float)
herit = master["herit_density_500m"].fillna(0).to_numpy().astype(float)
pop = master["pop_density_per_km2"].fillna(0).to_numpy().astype(float)
log_rent = master["log_rent"].fillna(master["log_rent"].median()).to_numpy().astype(float)
rent_missing = master["rent_missing"].fillna(1).to_numpy().astype(float)

# Pull fitted coefficients from statsmodels
# logit_model.params index should include 'const' and each standardized feature name
params = logit_model.params.to_dict()

# Ensure feature order matches JS computation
# These are the same as in your model:
# features = ["tau_min","dist_to_g_m","herit_density_500m","pop_density_per_km2","log_rent","rent_missing"]
coef = {
    "const": float(params.get("const", 0.0)),
    "tau_min": float(params.get("tau_min", 0.0)),
    "dist_to_g_m": float(params.get("dist_to_g_m", 0.0)),
    "herit_density_500m": float(params.get("herit_density_500m", 0.0)),
    "pop_density_per_km2": float(params.get("pop_density_per_km2", 0.0)),
    "log_rent": float(params.get("log_rent", 0.0)),
    "rent_missing": float(params.get("rent_missing", 0.0)),
}

# Standardization stats from estimation
m = {k: float(means[k]) for k in features}
s = {k: float(stds[k]) for k in features}

# g point in lon/lat and UTM (for separation)
g_lon, g_lat = G_LON, G_LAT
g_utm = gpd.GeoSeries([Point(g_lon, g_lat)], crs="EPSG:4326").to_crs("EPSG:32638").iloc[0]
g_utm_x, g_utm_y = float(g_utm.x), float(g_utm.y)

payload = {
    "geojson": json.loads(geojson_str),
    "cx": cx.tolist(),
    "cy": cy.tolist(),
    "dist_g": dist_g.tolist(),
    "herit": herit.tolist(),
    "pop": pop.tolist(),
    "log_rent": log_rent.tolist(),
    "rent_missing": rent_missing.tolist(),
    "coef": coef,
    "mean": m,
    "std": s,
    "mu0": {"x": float(mu0x), "y": float(mu0y)},
    "g": {"lon": float(g_lon), "lat": float(g_lat), "x": g_utm_x, "y": g_utm_y},
    "speed_m_per_min": float(SPEED_M_PER_MIN),
}

html_template = """
<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Yerevan sliders (continuous)</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"/>
  <style>
    body { margin:0; font-family: Arial, sans-serif; }
    #map { height: 86vh; width: 100%; }
    #controls {
      height: 14vh; padding: 10px 14px; box-sizing: border-box;
      display: grid; grid-template-columns: 1fr; gap: 8px;
      border-top: 1px solid #ddd;
    }
    .row { display:flex; gap:12px; align-items:center; }
    .label { width: 320px; }
    .value { font-weight: 600; width: 70px; }
    .info { margin-left: 8px; }
    .legend {
      background: white; padding: 10px; line-height: 1.2;
      border: 1px solid #ccc; border-radius: 6px;
    }
  </style>
</head>
<body>
  <div id="map"></div>
  <div id="controls">
    <div class="row">
      <div class="label">Transport factor (0.1..2.0)</div>
      <div class="value" id="tVal"></div>
      <input id="tSlider" class="vSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00" orient="vertical">
      <div class="info">Lower = faster</div>
    </div>

    <div class="row">
      <div class="label">Historic amenity strength (0.1..2.0)</div>
      <div class="value" id="aVal"></div>
      <input id="aSlider" class="vSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00" orient="vertical">
      <div class="info">Higher = stronger</div>
    </div>

    <div class="row">
      <div class="label">μ and separation</div>
      <div class="info" id="muInfo"></div>
    </div>
  </div>

  <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/proj4js/2.9.0/proj4.js"></script>

  <script>
    const data = __PAYLOAD__;

    // Define EPSG:32638 for proj4 (UTM zone 38N, WGS84)
    proj4.defs("EPSG:32638", "+proj=utm +zone=38 +datum=WGS84 +units=m +no_defs");

    const cx = data.cx;
    const cy = data.cy;
    const dist_g = data.dist_g;
    const herit = data.herit;
    const pop = data.pop;
    const log_rent = data.log_rent;
    const rent_missing = data.rent_missing;

    const coef = data.coef;
    const mean = data.mean;
    const std = data.std;

    const speed_m_per_min = data.speed_m_per_min;

    const g = data.g; // lon/lat + utm x/y
    const mu0 = data.mu0;

    function sigmoid(z) {
      // avoid overflow
      if (z > 35) return 1.0;
      if (z < -35) return 0.0;
      return 1.0 / (1.0 + Math.exp(-z));
    }

    function colorRamp(s) {
      s = Math.max(0, Math.min(1, s));
      const r = Math.round(255 * s);
      const gg = 60;
      const b = Math.round(255 * (1 - s));
      return `rgb(${r},${gg},${b})`;
    }

    function z(x, m, s) {
      if (!isFinite(x)) return 0.0;
      if (s === 0) return 0.0;
      return (x - m) / s;
    }

    // Fixed-point solver: given (tFactor, aFactor) compute μ and shares
    function solve(tFactor, aFactor) {
      let mux = mu0.x;
      let muy = mu0.y;

      const shares = new Array(cx.length).fill(0.0);

      for (let it = 0; it < 20; it++) {
        let sumw = 0.0, sumx = 0.0, sumy = 0.0;

        for (let i = 0; i < cx.length; i++) {
          const dx = cx[i] - mux;
          const dy = cy[i] - muy;
          const d = Math.sqrt(dx*dx + dy*dy);

          const tau = tFactor * (d / speed_m_per_min);

          // Standardize features using training stats
          const z_tau   = z(tau, mean.tau_min, std.tau_min);
          const z_dg    = z(dist_g[i], mean.dist_to_g_m, std.dist_to_g_m) * aFactor;
          const z_herit = z(herit[i], mean.herit_density_500m, std.herit_density_500m) * aFactor;
          const z_pop   = z(pop[i], mean.pop_density_per_km2, std.pop_density_per_km2);
          const z_lr    = z(log_rent[i], mean.log_rent, std.log_rent);
          const z_rm    = z(rent_missing[i], mean.rent_missing, std.rent_missing);

          const lin =
            coef.const +
            coef.tau_min * z_tau +
            coef.dist_to_g_m * z_dg +
            coef.herit_density_500m * z_herit +
            coef.pop_density_per_km2 * z_pop +
            coef.log_rent * z_lr +
            coef.rent_missing * z_rm;

          const s = sigmoid(lin);
          shares[i] = s;

          sumw += s;
          sumx += s * cx[i];
          sumy += s * cy[i];
        }

        const muxNew = sumx / sumw;
        const muyNew = sumy / sumw;
        const shift = Math.sqrt((muxNew - mux)**2 + (muyNew - muy)**2);

        mux = muxNew;
        muy = muyNew;
        if (shift < 30.0) break;
      }

      return { mux, muy, shares };
    }

    // Convert μ from UTM to lon/lat for Leaflet marker
    function utmToLonLat(x, y) {
      const out = proj4("EPSG:32638", "EPSG:4326", [x, y]);
      return { lon: out[0], lat: out[1] };
    }

    // Map init
    const map = L.map('map').setView([g.lat, g.lon], 11);
    L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
      maxZoom: 19,
      attribution: '&copy; OpenStreetMap contributors'
    }).addTo(map);

    L.marker([g.lat, g.lon]).addTo(map).bindPopup("g (historic center)");

    let muMarker = L.circleMarker([g.lat, g.lon], {
      radius: 7, weight: 2, color: '#000', fillColor: '#fff', fillOpacity: 1
    }).addTo(map).bindPopup("μ (economic center)");

    // GeoJSON layer
    const gridLayer = L.geoJSON(data.geojson, {
      style: function(feature) {
        return { fillColor:"#ddd", color:"#666", weight:0.2, fillOpacity:0.65 };
      }
    }).addTo(map);

    // Debounce updates so dragging slider is smooth
    let pending = null;

    function applyResult(res) {
      const muLL = utmToLonLat(res.mux, res.muy);
      muMarker.setLatLng([muLL.lat, muLL.lon]);

      // compute separation in meters from g in UTM
      const sep = Math.sqrt((res.mux - g.x)**2 + (res.muy - g.y)**2);

      document.getElementById('muInfo').textContent =
        `μ: (${muLL.lon.toFixed(5)}, ${muLL.lat.toFixed(5)})  |  |μ-g|: ${sep.toFixed(0)} m`;

      // Update styles using shares array
      gridLayer.setStyle(function(feature) {
        const id = feature.properties.gid;
        const s = res.shares[id];
        return {
          fillColor: colorRamp(s),
          color: '#666',
          weight: 0.2,
          fillOpacity: 0.65
        };
      });
    }

    function update() {
      const tVal = parseFloat(document.getElementById('tSlider').value);
      const aVal = parseFloat(document.getElementById('aSlider').value);

      document.getElementById('tVal').textContent = tVal.toFixed(2);
      document.getElementById('aVal').textContent = aVal.toFixed(2);

      const res = solve(tVal, aVal);
      applyResult(res);
    }

    function scheduleUpdate() {
      if (pending) clearTimeout(pending);
      pending = setTimeout(update, 120);
    }

    document.getElementById('tSlider').addEventListener('input', scheduleUpdate);
    document.getElementById('aSlider').addEventListener('input', scheduleUpdate);

    // Initial draw
    update();
  </script>
</body>
</html>
"""

html_filled = html_template.replace("__PAYLOAD__", json.dumps(payload))

html_path = os.path.join(OUT_DIR, "yerevan_continuous_two_sliders.html")
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html_filled)

html_path


'data/yerevan_interactive\\yerevan_continuous_two_sliders.html'

In [16]:
IFrame(src="data/yerevan_interactive/yerevan_continuous_two_sliders.html", width="100%", height=760)


In [33]:
from pathlib import Path
from IPython.display import IFrame, HTML, display

def write_interactive_html(out_path="thesis_dashboard_fit_one_screen.html"):
    html = r"""<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>Interactive thesis figures (3-up + compact controls)</title>

  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">

  <script src="https://cdn.plot.ly/plotly-2.30.0.min.js"></script>

  <style>
    :root{
      --pad: 12px;
      --gap: 10px;
      --line: rgba(0,0,0,0.12);
      --muted: rgba(0,0,0,0.62);

      /* Golden mean: cap plot heights so they do not stretch */
      --plotH: clamp(220px, 34vh, 320px);
      --plotHSmall: clamp(200px, 30vh, 280px);
    }

    html, body { height: 100%; }
    body {
      font-family: Inter, system-ui, -apple-system, Segoe UI, Roboto, sans-serif;
      margin: var(--pad);
      background: #fff;
      color: #111;
    }

    h2 { margin: 0 0 8px 0; font-size: 18px; }

    .grid3 {
      display: grid;
      grid-template-columns: repeat(3, minmax(240px, 1fr));
      gap: var(--gap);
      align-items: start;
      margin-bottom: var(--gap);
    }

    .card {
      border: 1px solid var(--line);
      border-radius: 14px;
      padding: 10px;
      background: #fff;
    }

    .plotTitle { font-weight: 750; margin: 0 0 4px 0; font-size: 13px; }
    .small { color: var(--muted); font-size: 11px; line-height: 1.25; margin: 0 0 6px 0; }
    .pill { display:inline-block; padding: 1px 7px; border: 1px solid var(--line); border-radius: 999px; font-size: 11px; margin-left: 6px; }

    /* Key change: explicit heights, not flex-fill */
    .plot { height: var(--plotH); }
    .plotSmall { height: var(--plotHSmall); }

    .controls {
      display: grid;
      grid-template-columns: 1fr 260px;
      gap: 12px;
      align-items: start;
    }

    .sliderRow {
      display: grid;
      grid-template-columns: 24px 1fr 72px;
      gap: 10px;
      align-items: center;
      margin: 6px 0;
    }
    .sliderRow .lbl { font-size: 12px; font-weight: 650; }
    .sliderRow .val { font-size: 12px; text-align: right; color: #222; }
    input[type="range"] { width: 100%; }

    .kvs { font-size: 12px; line-height: 1.25; }
    .kvs b { font-weight: 650; }
    hr { border: 0; border-top: 1px solid rgba(0,0,0,0.08); margin: 8px 0; }

    @media (max-width: 1050px) {
      .grid3 { grid-template-columns: 1fr; }
      .controls { grid-template-columns: 1fr; }
      :root{
        --plotH: clamp(240px, 38vh, 360px);
        --plotHSmall: clamp(220px, 34vh, 320px);
      }
    }
  </style>
</head>

<body>
  <h2>Interactive figures: 3 plots in a row, compact sliders below</h2>

  <div class="grid3">
    <div class="card">
      <div class="plotTitle">City layout <span class="pill">instant response</span></div>
      <div class="small">
        Thick segment is the business area. The x marker is the historic center (g). The circle is the economic center (μ).
      </div>
      <div id="plot_layout" class="plot plotSmall"></div>
    </div>

    <div class="card">
      <div class="plotTitle">Figure 1 style: sweep historic center location</div>
      <div class="small">
        We sweep g from 0 to 1.0 (x axis). Dashed line is the current g from the slider.
        Green X markers show current q and p at that g.
      </div>
      <div id="plot_fig1" class="plot"></div>
    </div>

    <div class="card">
      <div class="plotTitle">Figure 2-3 style: sweep transport cost</div>
      <div class="small">
        We sweep t from 0.0 to 1.5 (y axis). Dashed vertical line marks g. Dots show the current t.
      </div>
      <div id="plot_fig2" class="plot"></div>
    </div>
  </div>

  <div class="card">
    <div class="controls">
      <div>
        <div class="plotTitle">Common parameters</div>
        <div class="small">These sliders update all three plots.</div>

        <div class="sliderRow">
          <div class="lbl">N</div>
          <input id="N" type="range" min="0.20" max="1.90" step="0.05" value="1.40"/>
          <div class="val" id="N_val"></div>
        </div>

        <div class="sliderRow">
          <div class="lbl">e</div>
          <input id="e" type="range" min="0.01" max="1.00" step="0.01" value="0.10"/>
          <div class="val" id="e_val"></div>
        </div>

        <div class="sliderRow">
          <div class="lbl">g</div>
          <input id="g" type="range" min="0.00" max="1.00" step="0.01" value="0.25"/>
          <div class="val" id="g_val"></div>
        </div>

        <div class="sliderRow">
          <div class="lbl">t</div>
          <input id="t" type="range" min="0.00" max="1.50" step="0.01" value="0.25"/>
          <div class="val" id="t_val"></div>
        </div>

        <div class="small" style="margin-top:6px;">
          Tip: Increasing t makes distance more painful, so the business area often shifts to reduce travel.
        </div>
      </div>

      <div>
        <div class="plotTitle">Current interpretation</div>
        <div class="kvs">
          <div style="margin-top:4px;"><b>Business area</b></div>
          <div>Left edge: <span id="q_out"></span></div>
          <div>Right edge: <span id="p_out"></span></div>

          <div style="margin-top:6px;"><b>Historic center</b>: <span id="g_out"></span></div>
          <div><b>Economic center</b>: <span id="mu_out"></span></div>

          <hr/>

          <div><b>Situation label</b></div>
          <div id="case_out" class="small" style="margin:0;"></div>
        </div>
      </div>
    </div>
  </div>

<script>
  const EPS = 1e-9;

  function clip(x, lo, hi) { return Math.max(lo, Math.min(hi, x)); }
  function fmt(x) {
    if (!isFinite(x)) return "nan";
    const ax = Math.abs(x);
    if (ax >= 10) return x.toFixed(3);
    return x.toFixed(4);
  }

  function elH(id, fallback){
    const el = document.getElementById(id);
    const h = el ? el.clientHeight : 0;
    return (h && h > 50) ? h : fallback;
  }

  function solveBounds(N, t, e, g) {
    if (!(N > 0 && N < 2)) return { ok:false, reason:"N must be between 0 and 2." };
    if (!(e > 0)) return { ok:false, reason:"e must be positive." };
    if (!(t >= 0)) return { ok:false, reason:"t must be zero or positive." };

    const M = 2 - N;
    const theta = t / e;

    function inCity(q, p) {
      return (q < p) && (q >= -1 - 1e-6) && (p <= 1 + 1e-6);
    }

    let muB = NaN, pB = NaN, qB = NaN;
    if (theta > EPS) {
      muB = -M * N / (4 * theta);
      pB = muB + M / 2;
      qB = muB - M / 2;
    }

    let muC = NaN, pC = NaN, qC = NaN;
    if (theta > EPS) {
      muC =  M * N / (4 * theta);
      pC = muC + M / 2;
      qC = muC - M / 2;
    }

    let muA = NaN, pA = NaN, qA = NaN;
    const denom = (N - 2 * theta);
    if (Math.abs(denom) > EPS) {
      muA = N * g / denom;
      pA = muA + M / 2;
      qA = muA - M / 2;
    }

    function scoreCase(kind, q, p, mu) {
      if (!isFinite(q) || !isFinite(p) || !isFinite(mu)) return null;
      let score = 0.0;

      if (!inCity(q, p)) {
        score += 10.0 * (Math.max(0.0, (-1 - q)) + Math.max(0.0, (p - 1)));
      }
      if (kind === "A") {
        score += Math.max(0.0, q - g) + Math.max(0.0, g - p);
      } else if (kind === "B") {
        score += Math.max(0.0, p - g);
      } else if (kind === "C") {
        score += Math.max(0.0, g - q);
      }
      return score;
    }

    const candidates = [];
    const sA = scoreCase("A", qA, pA, muA);
    if (sA !== null) candidates.push([sA, "Historic center sits inside the business area.", qA, pA, muA]);
    const sB = scoreCase("B", qB, pB, muB);
    if (sB !== null) candidates.push([sB, "Historic center is to the right of the business area.", qB, pB, muB]);
    const sC = scoreCase("C", qC, pC, muC);
    if (sC !== null) candidates.push([sC, "Historic center is to the left of the business area.", qC, pC, muC]);

    if (candidates.length === 0) return { ok:false, reason:"No valid configuration found for these values." };

    candidates.sort((a,b) => a[0] - b[0]);
    const best = candidates[0];

    return {
      ok: true,
      N, M, t, e, g, theta,
      caseText: best[1],
      q: best[2],
      p: best[3],
      mu: best[4]
    };
  }

  const state = { N: 1.40, e: 0.10, g: 0.25, t: 0.25 };

  function readSlider(id) {
    return parseFloat(document.getElementById(id).value);
  }

  function syncLabelsFromState() {
    document.getElementById("N_val").textContent = fmt(state.N);
    document.getElementById("e_val").textContent = fmt(state.e);
    document.getElementById("g_val").textContent = fmt(state.g);
    document.getElementById("t_val").textContent = fmt(state.t);
  }

  function updateText(sol) {
    if (!sol.ok) {
      document.getElementById("q_out").textContent = "nan";
      document.getElementById("p_out").textContent = "nan";
      document.getElementById("g_out").textContent = "nan";
      document.getElementById("mu_out").textContent = "nan";
      document.getElementById("case_out").textContent = sol.reason;
      return;
    }
    document.getElementById("q_out").textContent = fmt(sol.q);
    document.getElementById("p_out").textContent = fmt(sol.p);
    document.getElementById("g_out").textContent = fmt(sol.g);
    document.getElementById("mu_out").textContent = fmt(sol.mu);
    document.getElementById("case_out").textContent = sol.caseText;
  }

  function drawLayout(sol) {
    let q = NaN, p = NaN, mu = NaN, g = NaN;
    if (sol.ok) { q = sol.q; p = sol.p; mu = sol.mu; g = sol.g; }

    const qd = clip(q, -1, 1);
    const pd = clip(p, -1, 1);

    const data = [
      { x: [-1, 1], y: [0, 0], mode: "lines", line: { width: 6 }, name: "city" },
      { x: sol.ok ? [qd, pd] : [], y: sol.ok ? [0, 0] : [], mode: "lines",
        line: { width: 18 }, opacity: 0.25, name: "business area" },
      { x: sol.ok ? [g] : [], y: sol.ok ? [0] : [], mode: "markers+text",
        marker: { size: 12, symbol: "x" }, text: ["g"], textposition: "top center", name: "historic center" },
      { x: sol.ok ? [mu] : [], y: sol.ok ? [0] : [], mode: "markers+text",
        marker: { size: 10, symbol: "circle" }, text: ["μ"], textposition: "top center", name: "economic center" },
      { x: sol.ok ? [q, p] : [], y: sol.ok ? [0, 0] : [], mode: "markers+text",
        marker: { size: 14, symbol: "line-ns-open" }, text: ["q", "p"], textposition: "bottom center", name: "edges" }
    ];

    const layout = {
      font: { size: 11 },
      margin: { l: 34, r: 10, t: 4, b: 26 },
      xaxis: { range: [-1.05, 1.05], title: "location on the city line" },
      yaxis: { range: [-0.8, 0.8], visible: false },
      showlegend: false,
      height: elH("plot_layout", 260)
    };

    Plotly.react("plot_layout", data, layout, {displayModeBar: false, responsive: false});
  }

  function drawFig1() {
    const N = state.N, e = state.e, t = state.t;
    const gmax = 1.0;
    const n = 320;

    const gs = [];
    const qs = [];
    const ps = [];

    for (let i=0; i<n; i++) {
      const g = gmax * (i/(n-1));
      const sol = solveBounds(N, t, e, g);
      gs.push(g);
      qs.push(sol.ok ? sol.q : NaN);
      ps.push(sol.ok ? sol.p : NaN);
    }

    const gCur = state.g;
    const solCur = solveBounds(N, t, e, gCur);
    const qCur = solCur.ok ? solCur.q : NaN;
    const pCur = solCur.ok ? solCur.p : NaN;

    const yLo = -1.05, yHi = 1.05;

    const data = [
      { x: gs, y: gs, mode: "lines", name: "g (reference)" },
      { x: gs, y: ps, mode: "lines", name: "right edge p(g)" },
      { x: gs, y: qs, mode: "lines", name: "left edge q(g)" },

      { x: gs, y: qs, mode: "lines", line: { width: 0 }, showlegend: false },
      { x: gs, y: ps, mode: "lines", fill: "tonexty", opacity: 0.2, name: "business area" },

      { x: [gCur, gCur], y: [yLo, yHi], mode: "lines",
        line: { dash: "dash", width: 2 }, name: "current g" },

      { x: solCur.ok ? [gCur, gCur] : [], y: solCur.ok ? [qCur, pCur] : [],
        mode: "markers",
        marker: { symbol: "x", size: 11, color: "green", line: { width: 2, color: "green" } },
        name: "current q,p" }
    ];

    const layout = {
      font: { size: 11 },
      margin: { l: 52, r: 10, t: 26, b: 38 },
      xaxis: { range: [0, gmax], title: "g (historic center location)" },
      yaxis: { range: [yLo, yHi], title: "location on the city line" },
      legend: {
        orientation: "h",
        yanchor: "bottom",
        y: 1.01,
        xanchor: "left",
        x: 0,
        font: { size: 10 }
      },
      height: elH("plot_fig1", 300)
    };

    Plotly.react("plot_fig1", data, layout, {displayModeBar: false, responsive: false});
  }

  function drawFig2() {
    const N = state.N, e = state.e, g = state.g;
    const tmin = 0.0;
    const tmax = 1.5;

    const n = 280;
    const ts = [];
    const qs = [];
    const ps = [];

    for (let i=0; i<n; i++) {
      const t = tmin + (tmax - tmin) * (i/(n-1));
      const sol = solveBounds(N, t, e, g);
      ts.push(t);
      qs.push(sol.ok ? sol.q : NaN);
      ps.push(sol.ok ? sol.p : NaN);
    }

    const solNow = solveBounds(N, state.t, e, g);
    const qNow = solNow.ok ? solNow.q : NaN;
    const pNow = solNow.ok ? solNow.p : NaN;

    const data = [
      { x: qs, y: ts, mode: "lines", name: "left edge q(t)" },
      { x: ps, y: ts, mode: "lines", name: "right edge p(t)" },

      { x: qs, y: ts, mode: "lines", line: { width: 0 }, showlegend: false },
      { x: ps, y: ts, mode: "lines", fill: "tonextx", opacity: 0.2, name: "business area" },

      { x: [g, g], y: [tmin, tmax], mode: "lines", line: { dash: "dash" }, name: "g (fixed)" },

      { x: [qNow, pNow], y: [state.t, state.t], mode: "markers", marker: { size: 8 }, name: "current t points" }
    ];

    const layout = {
      font: { size: 11 },
      margin: { l: 52, r: 10, t: 26, b: 38 },
      xaxis: { range: [-1.05, 1.05], title: "location on the city line" },
      yaxis: { range: [tmin, tmax], title: "t (transport cost)" },
      legend: {
        orientation: "h",
        yanchor: "bottom",
        y: 1.01,
        xanchor: "left",
        x: 0,
        font: { size: 10 }
      },
      height: elH("plot_fig2", 300)
    };

    Plotly.react("plot_fig2", data, layout, {displayModeBar: false, responsive: false});
  }

  function updateAll() {
    const sol = solveBounds(state.N, state.t, state.e, state.g);
    updateText(sol);
    drawLayout(sol);
    drawFig1();
    drawFig2();
  }

  function wireUp() {
    syncLabelsFromState();
    updateAll();

    ["N","e","g","t"].forEach((id) => {
      const el = document.getElementById(id);
      el.addEventListener("input", () => {
        state[id] = readSlider(id);
        syncLabelsFromState();
        updateAll();
      });
    });

    let rT = 0;
    window.addEventListener("resize", () => {
      clearTimeout(rT);
      rT = setTimeout(() => updateAll(), 120);
    });
  }

  wireUp();
</script>

</body>
</html>
"""
    out_path = Path(out_path)
    out_path.write_text(html, encoding="utf-8")
    return out_path

path = write_interactive_html("./data/yerevan_interactive/thesis_sectioned_dashboard.html")

try:
    display(IFrame(src=str(path), width=1400, height=860))
except Exception:
    display(HTML(path.read_text(encoding="utf-8")))

print(f"Wrote: {path.resolve()}")


Wrote: C:\Users\Nedric\Armenia\data\yerevan_interactive\thesis_sectioned_dashboard.html


In [35]:
import os
import re
import html as _html
import urllib.parse
from string import Template
from pathlib import Path


# =========================
# Config
# =========================
MODEL_FOCUS_MODE = "remove_zoom_effect"  # kept for compatibility


# =========================
# Small helpers
# =========================
def svg_placeholder_data_uri(label: str, w: int = 1600, h: int = 1000) -> str:
    safe = _html.escape(label)
    svg = f"""<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}" viewBox="0 0 {w} {h}">
  <defs>
    <linearGradient id="g" x1="0" y1="0" x2="1" y2="1">
      <stop offset="0" stop-color="#f2f2f2"/>
      <stop offset="1" stop-color="#dcdcdc"/>
    </linearGradient>
  </defs>
  <rect width="{w}" height="{h}" rx="48" fill="url(#g)"/>
  <rect x="48" y="48" width="{w-96}" height="{h-96}" rx="36" fill="none" stroke="#bdbdbd" stroke-width="4"/>
  <text x="50%" y="52%" text-anchor="middle" font-family="Inter, Arial, sans-serif" font-size="54" fill="#333">{safe}</text>
  <text x="50%" y="59%" text-anchor="middle" font-family="Inter, Arial, sans-serif" font-size="24" fill="#666">
    Replace with assets/ image
  </text>
</svg>"""
    return "data:image/svg+xml;charset=utf-8," + urllib.parse.quote(svg)


def _escape(s: str) -> str:
    return _html.escape(str(s), quote=True)


# =========================
# Patch 1: scrolly control hooks for iframe
# =========================
def patch_interactive_for_scrolly(interactive_html: str) -> str:
    injection = r"""
    // ---- scrollytelling control hooks (injected) ----
    function _dispatchInput(el) {
      try {
        el.dispatchEvent(new Event("input", { bubbles: true }));
        el.dispatchEvent(new Event("change", { bubbles: true }));
      } catch (e) {}
    }

    function setSliders(tVal, aVal) {
      const tSlider = document.getElementById("tSlider");
      const aSlider = document.getElementById("aSlider");
      if (!tSlider || !aSlider) return;

      if (typeof tVal === "number" && isFinite(tVal)) tSlider.value = String(tVal);
      if (typeof aVal === "number" && isFinite(aVal)) aSlider.value = String(aVal);

      _dispatchInput(tSlider);
      _dispatchInput(aSlider);

      try {
        if (typeof update === "function") update();
      } catch (e) {}
    }

    function setMapView(lat, lon, zoom) {
      if (typeof lat === "number" && typeof lon === "number" && typeof zoom === "number") {
        try {
          map.setView([lat, lon], zoom, { animate: true });
        } catch (e) {}
      }
    }

    let _invT = 0;
    function invalidateMapSize() {
      try {
        clearTimeout(_invT);
        _invT = setTimeout(() => map.invalidateSize(), 0);
      } catch (e) {}
    }

    function postReady() {
      try {
        window.parent && window.parent.postMessage({ type: "vizReady" }, "*");
      } catch (e) {}
    }

    window.addEventListener("message", (event) => {
      const msg = event.data;
      if (!msg || typeof msg !== "object") return;

      if (msg.type === "setSliders") setSliders(msg.t, msg.a);
      if (msg.type === "setView") setMapView(msg.lat, msg.lon, msg.zoom);
      if (msg.type === "invalidateSize") invalidateMapSize();

      if (msg.type === "ping") postReady();
    });

    postReady();
    // ---- end injected block ----
    """

    anchor = "    // Initial draw"
    if anchor in interactive_html:
        return interactive_html.replace(anchor, injection + "\n" + anchor, 1)

    idx = interactive_html.rfind("</script>")
    if idx == -1:
        raise ValueError("Could not find </script> to inject scrolly hooks.")
    return interactive_html[:idx] + injection + "\n" + interactive_html[idx:]


# =========================
# Patch: landing CSS
# Goal:
# - make ONLY the #model section use near-full width (small margins)
# - make the left scrollytelling column compact (fixed max width)
# - make the map column take the rest
# - keep nav always visible (fixed)
# - show minibar only while sticky is pinned, and push map down so no overlap
# =========================
def patch_landing_for_model_focus_zoom(landing_html: str, focus_mode: str = "remove_zoom_effect") -> str:
    css_inject = r"""

/* ---- injected: make model section full width + compact steps + no minibar overlap ---- */

:root{
  --navH: 68px;     /* JS overwrites */
  --miniBarH: 0px;  /* becomes >0 only while pinned */
  --modelPad: 12px; /* side padding for model section */
  --gapTop: 10px;
  --stepsW: 360px;  /* compact annotation width */
}

/* Top menu always visible */
.nav{
  position: fixed !important;
  top: 0 !important;
  left: 0 !important;
  right: 0 !important;
  z-index: 300 !important;
}

/* Page starts below fixed nav */
body{
  padding-top: var(--navH) !important;
}

/* Keep anchor jumps from hiding under nav */
#gallery1, #model, #gallery2, #explain{
  scroll-margin-top: calc(var(--navH) + 16px) !important;
}

/* Make only the model section nearly full width with small margins */
#model .container{
  max-width: none !important;
  width: calc(100vw - (2 * var(--modelPad))) !important;
  padding-left: var(--modelPad) !important;
  padding-right: var(--modelPad) !important;
  margin-left: auto !important;
  margin-right: auto !important;
}

/* The header inside model should also align with new width */
#model .modelHeader{
  padding-left: 0 !important;
  padding-right: 0 !important;
}

/* Force the model grid to: compact left, huge right */
#model .modelGrid{
  display: grid !important;
  grid-template-columns: var(--stepsW) 1fr !important;
  gap: 18px !important;
  align-items: start !important;
}

/* Ensure the right column can expand fully */
#model .stickyViz{
  min-width: 0 !important;
}

/* Compact the steps column */
#model .stepsCol{
  width: var(--stepsW) !important;
  max-width: var(--stepsW) !important;
  padding-right: 0 !important;
  opacity: 0.98 !important;
}

/* Make step cards compact */
#model .stepCard{
  max-width: var(--stepsW) !important;
  padding: 12px 12px 10px 12px !important;
  border-radius: 20px !important;
}
#model .stepKicker{
  font-size: 10px !important;
}
#model .stepCard h3{
  font-size: 16px !important;
  margin-bottom: 6px !important;
}
#model .stepBody{
  font-size: 13px !important;
  line-height: 1.45 !important;
}
#model .chip{
  font-size: 11px !important;
  padding: 5px 9px !important;
}

/* Sticky map: always stick below nav, plus minibar when pinned */
#model .stickyViz{
  position: sticky !important;
  top: calc(var(--navH) + var(--miniBarH) + var(--gapTop)) !important;
  height: calc(100vh - var(--navH) - var(--miniBarH) - (2 * var(--gapTop))) !important;
  transition: none !important;
}

/* Hide minibar by default */
#model .modelMiniBar{
  display: none !important;
}

/* Show minibar only while pinned, below nav */
body.modelPinned{
  --miniBarH: 56px;
}

body.modelPinned #model .modelMiniBar{
  display: flex !important;
  position: fixed !important;
  top: var(--navH) !important;
  left: 0 !important;
  right: 0 !important;
  z-index: 250 !important;
  background: rgba(250,250,250,.94) !important;
  backdrop-filter: blur(10px) !important;
  border-bottom: 1px solid rgba(15,15,16,.12) !important;
  padding: 8px 18px 12px 18px !important;
}

/* Extra safety: when pinned, keep stickyViz pushed down */
body.modelPinned #model .stickyViz{
  top: calc(var(--navH) + var(--miniBarH) + var(--gapTop)) !important;
  height: calc(100vh - var(--navH) - var(--miniBarH) - (2 * var(--gapTop))) !important;
}

/* Smaller overall section paddings can help the map feel bigger */
#model.modelWrap, .modelWrap{
  padding-left: 0 !important;
  padding-right: 0 !important;
}

/* Mobile */
@media (max-width: 980px){
  :root{
    --stepsW: 100%;
    --modelPad: 12px;
  }

  #model .modelGrid{
    grid-template-columns: 1fr !important;
  }

  #model .stickyViz{
    position: relative !important;
    top: 0 !important;
    height: 70vh !important;
  }

  body.modelPinned{
    --miniBarH: 54px;
  }
}

/* ---- end injected block ---- */
"""

    style_close = landing_html.rfind("</style>")
    if style_close == -1:
        raise ValueError("Could not find </style> to inject CSS.")
    return landing_html[:style_close] + css_inject + "\n" + landing_html[style_close:]


def patch_center_map_button_zoom(viz_html: str, center_zoom: float = 15.0) -> str:
    needle = "map.setView([g.lat, g.lon], 11, { animate: true });"
    replacement = f"map.setView([g.lat, g.lon], {center_zoom}, {{ animate: true }});"
    if needle not in viz_html:
        raise ValueError("Could not find the Center map setView call to patch.")
    return viz_html.replace(needle, replacement, 1)


# =========================
# Patch 2: iframe UI layout (shrink side panels so the map is bigger)
# =========================
def patch_interactive_ui_left_right_vertical_sliders(html_str: str) -> str:
    old_body_pattern = re.compile(
        r'<div id="map"></div>\s*<div id="controls">.*?</div>\s*',
        re.DOTALL
    )

    new_body = r"""
  <div id="app">
    <div id="topRow">

      <div class="panel" id="leftPanel">
        <div class="panelKicker">Transport</div>
        <div class="panelTitle">Transport factor</div>

        <div class="panelMetaRow">
          <div class="panelRange">0.1 .. 2.0</div>
        </div>

        <div class="panelSliderWrap">
          <input id="tSlider" class="vSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00">
          <div class="vSliderValueCol">
            <div class="panelValue" id="tVal"></div>
          </div>
        </div>

        <div class="panelFoot">Lower means faster</div>
      </div>

      <div class="mapPanel" id="mapPanel">
        <div id="map"></div>
      </div>

      <div class="panel" id="rightPanel">
        <div class="panelKicker">Amenities</div>
        <div class="panelTitle">Historic amenity strength</div>

        <div class="panelMetaRow">
          <div class="panelRange">0.1 .. 2.0</div>
        </div>

        <div class="panelSliderWrap">
          <input id="aSlider" class="vSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00">
          <div class="vSliderValueCol">
            <div class="panelValue" id="aVal"></div>
          </div>
        </div>

        <div class="panelFoot">Higher means stronger</div>
      </div>

    </div>

    <div id="bottomBar">
      <div class="barLeft">
        <div class="barLabel">Distance</div>
        <button id="centerBtn" class="barBtn" type="button">Center map</button>
      </div>
      <div class="barValue" id="muInfo"></div>
    </div>
  </div>
"""
    if not old_body_pattern.search(html_str):
        raise ValueError("Could not find the expected map+controls block to replace.")
    html_str = old_body_pattern.sub(new_body, html_str, count=1)

    style_pattern = re.compile(r"<style>.*?</style>", re.DOTALL)
    new_style = r"""
  <style>
    html, body { height: 100%; }
    body { margin: 0; font-family: Arial, sans-serif; overflow: hidden; background: #fff; }

    #app { height: 100%; display: flex; flex-direction: column; box-sizing: border-box; }

    #topRow {
      flex: 1;
      min-height: 0;
      display: grid;

      /* side panels smaller so map is bigger */
      grid-template-columns: clamp(92px, 7vw, 128px) 1fr clamp(92px, 7vw, 128px);

      gap: 10px;
      padding: 10px;
      box-sizing: border-box;
      background: #fff;
    }

    .panel {
      align-self: stretch;
      height: 100%;
      border: 1px solid rgba(0,0,0,0.14);
      border-radius: 16px;
      background: #fff;
      box-shadow: 0 10px 26px rgba(0,0,0,0.10);
      padding: 10px;
      box-sizing: border-box;
      display: flex;
      flex-direction: column;
      gap: 6px;
    }

    .panelKicker { font-size: 10px; letter-spacing: 0.12em; text-transform: uppercase; opacity: 0.65; }
    .panelTitle { font-weight: 800; font-size: 13px; margin-top: 2px; }

    .panelMetaRow {
      display: flex;
      justify-content: space-between;
      align-items: baseline;
      gap: 8px;
      margin-top: 2px;
    }

    .panelRange { font-size: 11px; opacity: 0.7; }

    .panelSliderWrap{
      flex: 1;
      display: flex;
      align-items: center;
      justify-content: center;
      gap: 12px;
      margin-top: 6px;
      min-height: 0;
    }

    .vSlider{
      -webkit-appearance: slider-vertical;
      appearance: slider-vertical;
      writing-mode: bt-lr;
      width: 18px;
      height: 100%;
      max-height: none;
      padding: 0;
      margin: 0;
    }

    .vSliderValueCol { align-self: center; }
    .panelValue { font-size: 13px; font-weight: 900; text-align: right; }

    .panelFoot { font-size: 11px; opacity: 0.7; margin-top: 6px; }

    .mapPanel {
      border: 1px solid rgba(0,0,0,0.14);
      border-radius: 16px;
      overflow: hidden;
      box-shadow: 0 10px 26px rgba(0,0,0,0.10);
      background: #f3f3f3;
      min-height: 0;
    }

    #map { width: 100%; height: 100%; }

    #bottomBar {
      border-top: 1px solid rgba(0,0,0,0.12);
      padding: 10px 12px;
      display: flex;
      justify-content: space-between;
      align-items: center;
      gap: 12px;
      background: #fff;
      box-sizing: border-box;
    }

    .barLeft { display: flex; align-items: center; gap: 10px; min-width: 0; }
    .barLabel { font-size: 11px; letter-spacing: 0.10em; text-transform: uppercase; opacity: 0.7; white-space: nowrap; }

    .barBtn {
      height: 30px;
      padding: 0 10px;
      border-radius: 999px;
      border: 1px solid rgba(0,0,0,0.18);
      background: rgba(255,255,255,0.92);
      font-size: 12px;
      font-weight: 700;
      cursor: pointer;
    }

    .barBtn:active { transform: translateY(1px); }

    .barValue { font-size: 16px; font-weight: 900; white-space: nowrap; }

    @media (max-width: 560px) {
      #topRow {
        grid-template-columns: 1fr;
        grid-template-rows: auto minmax(240px, 1fr) auto;
      }
      .panel { align-self: stretch; }
      .mapPanel { min-height: 45vh; }
      .vSlider { height: 120px; }
    }
  </style>
"""
    if not style_pattern.search(html_str):
        raise ValueError("Could not find a <style> block to replace.")
    html_str = style_pattern.sub(new_style, html_str, count=1)

    tile_add_pattern = re.compile(r"\}\)\.addTo\(map\);\s*", re.DOTALL)
    html_str = tile_add_pattern.sub(
        "}).addTo(map);\n\n    setTimeout(() => map.invalidateSize(), 0);\n\n    ",
        html_str,
        count=1
    )

    muinfo_pattern = re.compile(
        r"document\.getElementById\('muInfo'\)\.textContent\s*=\s*`.*?`;\s*",
        re.DOTALL
    )
    if muinfo_pattern.search(html_str):
        html_str = muinfo_pattern.sub(
            "document.getElementById('muInfo').textContent = `|μ - g|: ${sep.toFixed(0)} m`;\n\n      ",
            html_str,
                       count=1
        )

    map_init_hook = re.compile(
        r"const map\s*=\s*L\.map\('map'(?:,[^)]*)?\)\.setView\(\[g\.lat,\s*g\.lon\],\s*11\);\s*",
        re.DOTALL
    )

    def _inject_center(m: re.Match) -> str:
        original = m.group(0)
        add = (
            "\n    const centerBtn = document.getElementById('centerBtn');\n"
            "    if (centerBtn) {\n"
            "      centerBtn.addEventListener('click', () => {\n"
            "        map.setView([g.lat, g.lon], 11, { animate: true });\n"
            "      });\n"
            "    }\n"
        )
        return original + add

    if map_init_hook.search(html_str):
        html_str = map_init_hook.sub(_inject_center, html_str, count=1)

    return html_str


# =========================
# Landing (index.html) generator helpers
# =========================
def build_steps_html(steps):
    out = []
    for st in steps:
        title = _escape(st.get("title", ""))
        heading = _escape(st.get("heading", st.get("title", "Step")))
        body = _escape(st.get("body", ""))
        hint = _escape(st.get("hint", ""))

        t = float(st.get("t", 1.0))
        a = float(st.get("a", 1.0))

        view_lat = st.get("view_lat", "")
        view_lon = st.get("view_lon", "")
        view_zoom = st.get("view_zoom", "")

        hint_html = f"<p class='stepHint'>{hint}</p>" if hint else ""

        out.append(
            f"""
            <section class="step"
              data-title="{title}"
              data-t="{t:.2f}"
              data-a="{a:.2f}"
              data-view-lat="{view_lat}"
              data-view-lon="{view_lon}"
              data-view-zoom="{view_zoom}">
              <div class="stepCard">
                <div class="stepKicker">Scenario</div>
                <h3>{heading}</h3>
                <p class="stepBody">{body}</p>
                {hint_html}
                <div class="chipRow">
                  <span class="chip">Transport: {t:.2f}</span>
                  <span class="chip">Amenity: {a:.2f}</span>
                </div>
              </div>
            </section>
            """
        )
    return "\n".join(out)


def build_gallery_html(items, cols=3):
    cards = []
    for it in items:
        src = _escape(it["src"])
        cap = _escape(it.get("caption", ""))
        cap_html = f"<figcaption>{cap}</figcaption>" if cap else ""
        cards.append(
            f"""
            <figure class="imgCard">
              <img src="{src}" alt="{cap}" loading="lazy"/>
              {cap_html}
            </figure>
            """
        )
    return f"""<div class="imgGrid cols{int(cols)}">{''.join(cards)}</div>"""


def build_explain_blocks(blocks):
    out = []
    for b in blocks:
        h = _escape(b.get("heading", ""))
        p = _escape(b.get("body", ""))
        out.append(
            f"""
            <div class="textCard">
              <h3>{h}</h3>
              <p>{p}</p>
            </div>
            """
        )
    return "\n".join(out)


# =========================
# Dashboard page generator (unchanged)
# =========================
def build_dashboard_page_html(
    page_title: str,
    iframe_src: str,
    iframe_title: str = "Theoretical modelling dashboard",
    height_px: int = 900,  # fallback only
) -> str:
    return f"""<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>{_escape(page_title)}</title>

  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">

  <style>
    :root{{
      --bg: #fafafa;
      --panel: #ffffff;
      --text: #0f0f10;
      --muted: rgba(15,15,16,.68);
      --line: rgba(15,15,16,.12);
      --radius2: 24px;
      --padX: 22px;
      --navH: 68px; /* set by JS */
    }}

    html, body {{ height: 100%; }}
    body{{
      margin:0;
      font-family: Inter, Arial, sans-serif;
      background: var(--bg);
      color: var(--text);
      display: flex;
      flex-direction: column;
    }}
    a{{ color: inherit; text-decoration: none; }}

    .nav{{
      position: sticky;
      top: 0;
      z-index: 50;
      background: rgba(250,250,250,.86);
      backdrop-filter: blur(10px);
      border-bottom: 1px solid var(--line);
    }}

    /* full width */
    .container{{
      width: 100%;
      box-sizing: border-box;
      padding: 0 var(--padX);
    }}

    .navInner{{
      display:flex;
      align-items:center;
      justify-content:space-between;
      padding: 14px 0;
      gap: 16px;
    }}
    .brand{{ font-weight: 700; letter-spacing: -0.02em; }}

    .btn{{
      display:inline-flex;
      align-items:center;
      justify-content:center;
      height: 40px;
      padding: 0 14px;
      border-radius: 999px;
      border: 1px solid var(--line);
      background: rgba(255,255,255,.86);
      font-weight: 600;
      font-size: 13px;
      cursor: pointer;
    }}

    /* main fills remaining viewport height */
    main{{
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 18px 0 22px 0;
      min-height: 0;
    }}

    .meta{{
      flex: 0 0 auto;
      margin: 0 0 12px 0;
      color: var(--muted);
      font-size: 13px;
      line-height: 1.5;
    }}

    .card{{
      flex: 1 1 auto;
      min-height: 0;
      background: var(--panel);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      overflow: hidden;
    }}

    iframe{{
      width: 100%;
      height: 100%;
      border: 0;
      display: block;
      background: #fff;
    }}
  </style>
</head>

<body>
  <div class="nav" id="navEl">
    <div class="container">
      <div class="navInner">
        <div class="brand">{_escape(page_title)}</div>
        <a class="btn" href="index.html#explain">Back</a>
      </div>
    </div>
  </div>

  <main class="container">
    <div class="meta" id="metaEl">{_escape(iframe_title)}</div>
    <div class="card" id="cardEl">
      <iframe id="dashFrame" src="{_escape(iframe_src)}" title="{_escape(iframe_title)}" loading="eager"></iframe>
    </div>
  </main>

  <script>
    (function() {{
      const nav = document.getElementById("navEl");
      const meta = document.getElementById("metaEl");
      const card = document.getElementById("cardEl");

      function sync() {{
        const navH = nav ? (nav.getBoundingClientRect().height || 68) : 68;
        document.documentElement.style.setProperty("--navH", Math.round(navH) + "px");

        /* Ensure the card is always visible and not taller than the viewport */
        const vh = window.innerHeight || 800;
        const metaH = meta ? (meta.getBoundingClientRect().height || 0) : 0;

        /* main has padding top/bottom (18 + 22) = 40 */
        const pad = 40;
        const side = 0;

        const target = Math.max(520, Math.floor(vh - navH - metaH - pad - side));
        if (card) card.style.height = target + "px";
      }}

      window.addEventListener("resize", sync);
      setTimeout(sync, 0);
      setTimeout(sync, 250);
    }})();
  </script>
</body>
</html>
"""




# =========================
# Landing (index.html) generator
# =========================
def build_landing_html(config: dict, focus_mode: str = "remove_zoom_effect") -> str:
    # This class is not strictly required anymore (CSS targets #model directly),
    # but leaving it does not hurt.
    body_class_attr = ' class="modelLayoutFocus"'

    t = Template(r"""<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>${PAGE_TITLE}</title>

  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">

  <style>
    :root{
      --bg: #fafafa;
      --panel: #ffffff;
      --text: #0f0f10;
      --muted: rgba(15,15,16,.68);
      --line: rgba(15,15,16,.12);
      --shadow: 0 14px 40px rgba(0,0,0,.08);
      --radius2: 24px;
      --max: 1160px;
    }

    html{ scroll-behavior: smooth; }
    body{
      margin:0;
      font-family: Inter, Arial, sans-serif;
      background: var(--bg);
      color: var(--text);
    }
    a{ color: inherit; text-decoration: none; }
    .container{ max-width: var(--max); margin: 0 auto; padding: 0 22px; }

    .nav{
      position: sticky;
      top: 0;
      z-index: 50;
      background: rgba(250,250,250,.86);
      backdrop-filter: blur(10px);
      border-bottom: 1px solid var(--line);
    }
    .navInner{
      display:flex;
      align-items:center;
      justify-content:space-between;
      padding: 14px 0;
      gap: 16px;
    }
    .brand{ font-weight: 700; letter-spacing: -0.02em; }
    .navLinks{ display:flex; gap: 14px; flex-wrap: wrap; justify-content: flex-end; }
    .navLinks a{
      font-size: 13px;
      color: var(--muted);
      padding: 8px 10px;
      border-radius: 999px;
      border: 1px solid transparent;
    }
    .navLinks a:hover{
      color: var(--text);
      border-color: var(--line);
      background: rgba(255,255,255,.7);
    }

    .hero{
      padding: 62px 0 36px 0;
      border-bottom: 1px solid var(--line);
    }
    .heroGrid{
      display:grid;
      grid-template-columns: 1.1fr .9fr;
      gap: 22px;
      align-items: start;
    }
    .kicker{
      font-size: 12px;
      color: var(--muted);
      letter-spacing: .12em;
      text-transform: uppercase;
      margin-bottom: 14px;
    }
    h1{
      font-size: clamp(34px, 4.2vw, 60px);
      line-height: 1.02;
      letter-spacing: -0.04em;
      margin: 0 0 14px 0;
    }
    .sub{
      font-size: 16px;
      color: var(--muted);
      line-height: 1.6;
      max-width: 54ch;
      margin: 0 0 22px 0;
    }
    .ctaRow{ display:flex; gap: 10px; flex-wrap: wrap; align-items:center; }
    .btn{
      display:inline-flex;
      align-items:center;
      justify-content:center;
      height: 44px;
      padding: 0 16px;
      border-radius: 999px;
      border: 1px solid var(--line);
      background: rgba(255,255,255,.86);
      font-weight: 600;
      font-size: 13px;
    }
    .btnPrimary{
      background: #111;
      color: #fff;
      border-color: #111;
    }

    .heroCard{
      background: var(--panel);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      overflow:hidden;
      box-shadow: 0 1px 0 rgba(0,0,0,.04);
    }
    .heroCard img{ width:100%; height: 360px; object-fit: cover; display:block; }
    .heroMeta{ padding: 14px 14px 16px 14px; color: var(--muted); font-size: 13px; line-height: 1.5; }

    .section{ padding: 46px 0; }
    .sectionTitle{
      display:flex;
      justify-content: space-between;
      align-items: baseline;
      gap: 12px;
      margin-bottom: 18px;
    }
    .sectionTitle h2{ margin:0; font-size: 22px; letter-spacing: -0.02em; }
    .sectionTitle p{ margin:0; color: var(--muted); font-size: 13px; max-width: 58ch; line-height: 1.5; }

    .imgGrid{ display:grid; gap: 14px; }
    .imgGrid.cols3{ grid-template-columns: repeat(3, 1fr); }
    .imgGrid.cols2{ grid-template-columns: repeat(2, 1fr); }
    .imgCard{
      margin: 0;
      background: var(--panel);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      overflow:hidden;
      box-shadow: 0 1px 0 rgba(0,0,0,.04);
    }
    .imgCard img{ width: 100%; height: 280px; object-fit: cover; display:block; }
    .imgCard figcaption{ padding: 12px 14px; font-size: 13px; color: var(--muted); line-height: 1.4; }

    .modelWrap{
      border-top: 1px solid var(--line);
      border-bottom: 1px solid var(--line);
      background: #fff;
    }
    .modelHeader{ padding: 46px 0 14px 0; }
    .modelGrid{
      display:grid;
      grid-template-columns: 1fr 1.2fr;
      gap: 18px;
      padding-bottom: 48px;
    }
    .stepsCol{ padding-right: 10px; }

    .stickyViz{
      position: sticky;
      top: 74px;
      height: calc(100vh - 96px);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      overflow:hidden;
      background: #f3f3f3;
      box-shadow: var(--shadow);
    }
    .stickyViz iframe{ width: 100%; height: 100%; border: 0; display:block; background: #fff; }

    .modelMiniBar{
      display:flex;
      align-items:center;
      justify-content:space-between;
      gap: 14px;
      padding: 10px 0 16px 0;
      color: var(--muted);
      font-size: 13px;
    }
    .progress{
      height: 6px;
      flex: 1;
      background: rgba(0,0,0,.08);
      border-radius: 999px;
      overflow:hidden;
    }
    .progress > div{ height: 100%; width: 0%; background: #111; }

    .step{ padding: 24vh 0; }
    .step:first-child{ padding-top: 14vh; }
    .step:last-child{ padding-bottom: 14vh; }

    .stepCard{
      background: #fff;
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      padding: 16px 16px 14px 16px;
      box-shadow: 0 1px 0 rgba(0,0,0,.04);
      transition: transform .15s ease, box-shadow .15s ease, border-color .15s ease;
    }
    .step.is-active .stepCard{
      border-color: rgba(0,0,0,.55);
      box-shadow: 0 18px 46px rgba(0,0,0,.10);
      transform: translateY(-1px);
    }
    .stepKicker{ font-size: 11px; letter-spacing: .12em; text-transform: uppercase; color: var(--muted); margin-bottom: 8px; }
    .stepCard h3{ margin: 0 0 8px 0; font-size: 18px; letter-spacing: -0.02em; }
    .stepBody{ margin: 0 0 10px 0; color: rgba(0,0,0,.72); font-size: 14px; line-height: 1.55; }
    .stepHint{ margin: 0 0 10px 0; color: var(--muted); font-size: 13px; line-height: 1.5; }
    .chipRow{ display:flex; gap: 8px; flex-wrap: wrap; margin-top: 6px; }
    .chip{ display:inline-flex; padding: 6px 10px; border-radius: 999px; border: 1px solid var(--line); background: rgba(250,250,250,.9); font-size: 12px; color: rgba(0,0,0,.72); }

    .explainGrid{ display:grid; grid-template-columns: 1fr 1fr; gap: 14px; margin-top: 14px; }
    .textCard{
      background: var(--panel);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      padding: 18px 18px 16px 18px;
      box-shadow: 0 1px 0 rgba(0,0,0,.04);
    }
    .textCard h3{ margin: 0 0 8px 0; font-size: 16px; }
    .textCard p{ margin:0; color: rgba(0,0,0,.72); font-size: 14px; line-height: 1.6; }

    footer{ border-top: 1px solid var(--line); padding: 26px 0 40px 0; color: var(--muted); font-size: 13px; line-height: 1.5; }

    @media (max-width: 980px){
      .heroGrid{ grid-template-columns: 1fr; }
      .modelGrid{ grid-template-columns: 1fr; }
      .stickyViz{ position: relative; top: 0; height: 70vh; }
      .imgGrid.cols3{ grid-template-columns: 1fr; }
      .imgGrid.cols2{ grid-template-columns: 1fr; }
      .explainGrid{ grid-template-columns: 1fr; }
      .step{ padding: 16vh 0; }
    }
  </style>
</head>

<body id="top""" + body_class_attr + r""">
  <div class="nav">
    <div class="container">
      <div class="navInner">
        <div class="brand">${BRAND}</div>
        <div class="navLinks">
          <a href="#gallery1">Gallery</a>
          <a href="#model">Model</a>
          <a href="#gallery2">More</a>
          <a href="#explain">Explanation</a>
        </div>
      </div>
    </div>
  </div>

  <header class="hero">
    <div class="container">
      <div class="heroGrid">
        <div>
          <div class="kicker">${HERO_KICKER}</div>
          <h1>${HERO_TITLE}</h1>
          <p class="sub">${HERO_SUB}</p>
          <div class="ctaRow">
            <a class="btn btnPrimary" href="#model">${CTA_PRIMARY}</a>
            <a class="btn" href="#explain">${CTA_SECONDARY}</a>
          </div>
        </div>

        <div class="heroCard">
          <img src="${HERO_IMAGE}" alt="" loading="eager">
          <div class="heroMeta">${HERO_IMAGE_CAPTION}</div>
        </div>
      </div>
    </div>
  </header>

  <section class="section" id="gallery1">
    <div class="container">
      <div class="sectionTitle">
        <h2>${G1_TITLE}</h2>
        <p>${G1_SUB}</p>
      </div>
      ${GALLERY1_HTML}
    </div>
  </section>

  <section class="modelWrap" id="model">
    <div class="container">
      <div class="modelHeader">
        <div class="sectionTitle">
          <h2>${MODEL_TITLE}</h2>
          <p>${MODEL_SUB}</p>
        </div>

        <div class="modelMiniBar">
          <div id="stepName">Baseline</div>
          <div class="progress"><div id="progBar"></div></div>
        </div>
      </div>

      <div class="modelGrid">
        <div class="stepsCol">
          ${STEPS_HTML}
        </div>

        <div class="stickyViz">
          <iframe id="viz" src="${VIZ_FILENAME}" title="Interactive model" loading="lazy"></iframe>
        </div>
      </div>
    </div>
  </section>

  <section class="section" id="gallery2">
    <div class="container">
      <div class="sectionTitle">
        <h2>${G2_TITLE}</h2>
        <p>${G2_SUB}</p>
      </div>
      ${GALLERY2_HTML}
    </div>
  </section>

  <section class="section" id="explain">
    <div class="container">
      <div class="sectionTitle">
        <h2>${EXPLAIN_TITLE}</h2>
        <p>${EXPLAIN_SUB}</p>
      </div>
      <div class="explainGrid">
        ${EXPLAIN_BLOCKS}
      </div>
      ${EXTRA_EXPLAIN_HTML}
    </div>
  </section>

  <footer>
    <div class="container">
      ${FOOTER_TEXT}
    </div>
  </footer>

  <script src="https://unpkg.com/scrollama"></script>
  <script>
    const iframe = document.getElementById("viz");
    const steps = Array.from(document.querySelectorAll(".step"));
    const stepName = document.getElementById("stepName");
    const progBar = document.getElementById("progBar");
    const stickyVizEl = document.querySelector("#model .stickyViz");
    const navEl = document.querySelector(".nav");

    const pending = { sliders: null, view: null, invalidate: false };

    function _postToViz(msg) {
      if (!iframe || !iframe.contentWindow) return;
      iframe.contentWindow.postMessage(msg, "*");
    }

    function _remember(msg) {
      if (!msg || typeof msg !== "object") return;
      if (msg.type === "setSliders") pending.sliders = msg;
      if (msg.type === "setView") pending.view = msg;
      if (msg.type === "invalidateSize") pending.invalidate = true;
    }

    function replayPending() {
      if (!iframe || !iframe.contentWindow) return;
      _postToViz({ type: "ping" });
      if (pending.sliders) _postToViz(pending.sliders);
      if (pending.view) _postToViz(pending.view);
      if (pending.invalidate) _postToViz({ type: "invalidateSize" });
    }

    function sendToViz(msg) {
      if (!iframe || !iframe.contentWindow) return;
      _remember(msg);
      _postToViz(msg);
    }

    if (iframe) {
      iframe.addEventListener("load", () => {
        replayPending();
        sendToViz({ type: "invalidateSize" });
      });
    }

    function syncNavHeightVar() {
      if (!navEl) return;
      const h = navEl.getBoundingClientRect().height || 68;
      document.documentElement.style.setProperty("--navH", Math.round(h) + "px");
    }

    window.addEventListener("resize", () => {
      syncNavHeightVar();
      schedulePinnedCheck();
    });

    setTimeout(syncNavHeightVar, 0);

    let lastIdx = -1;
    function activateStep(el, idx) {
      if (!el) return;
      if (idx === lastIdx) return;
      lastIdx = idx;

      steps.forEach(s => s.classList.remove("is-active"));
      el.classList.add("is-active");

      const t = parseFloat(el.dataset.t);
      const a = parseFloat(el.dataset.a);

      const lat = parseFloat(el.dataset.viewLat);
      const lon = parseFloat(el.dataset.viewLon);
      const zoom = parseFloat(el.dataset.viewZoom);

      const title = el.dataset.title || ("Step " + String(idx + 1));
      if (stepName) stepName.textContent = title;

      if (isFinite(t) && isFinite(a)) sendToViz({ type: "setSliders", t: t, a: a });
      if (isFinite(lat) && isFinite(lon) && isFinite(zoom)) sendToViz({ type: "setView", lat: lat, lon: lon, zoom: zoom });

      const pct = steps.length <= 1 ? 100 : (idx / (steps.length - 1)) * 100;
      if (progBar) progBar.style.width = String(pct.toFixed(1)) + "%";
    }

    // Minibar: show only when sticky map is actually pinned
    let pinnedOn = false;

    function setPinned(on) {
      const next = !!on;
      if (pinnedOn === next) return;
      pinnedOn = next;
      document.body.classList.toggle("modelPinned", pinnedOn);

      setTimeout(() => {
        sendToViz({ type: "invalidateSize" });
        replayPending();
      }, 140);
    }

    function computePinned() {
      if (!stickyVizEl) { setPinned(false); return; }

      const cs = getComputedStyle(stickyVizEl);
      const pos = cs.position;
      if (pos !== "sticky" && pos !== "-webkit-sticky") { setPinned(false); return; }

      const r = stickyVizEl.getBoundingClientRect();
      const topPx = parseFloat(cs.top) || 0;
      const vh = window.innerHeight || 800;

      const delta = Math.abs(r.top - topPx);
      const pinned = pinnedOn ? (delta <= 26) : (delta <= 2);
      const visible = r.bottom > (topPx + 140) && r.top < (vh - 140);

      setPinned(pinned && visible);
    }

    let pinRAF = 0;
    function schedulePinnedCheck() {
      cancelAnimationFrame(pinRAF);
      pinRAF = requestAnimationFrame(computePinned);
    }

    window.addEventListener("scroll", schedulePinnedCheck, { passive: true });

    if (stickyVizEl && "ResizeObserver" in window) {
      const ro = new ResizeObserver(() => {
        setTimeout(() => sendToViz({ type: "invalidateSize" }), 140);
      });
      ro.observe(stickyVizEl);
    }

    const scroller = scrollama();
    scroller
      .setup({ step: ".step", offset: 0.62 })
      .onStepEnter((resp) => {
        schedulePinnedCheck();
        activateStep(resp.element, resp.index);
      })
      .onStepExit((resp) => {
        if (resp.direction === "up" && resp.index > 0) {
          const prev = resp.index - 1;
          activateStep(steps[prev], prev);
        }
        schedulePinnedCheck();
      });

    window.addEventListener("message", (event) => {
      if (event.data && event.data.type === "vizReady") {
        replayPending();
        sendToViz({ type: "invalidateSize" });
      }
    });

    // Init
    syncNavHeightVar();
    schedulePinnedCheck();
    setTimeout(() => {
      if (steps.length) activateStep(steps[0], 0);
      replayPending();
      sendToViz({ type: "invalidateSize" });
      schedulePinnedCheck();
    }, 700);
  </script>
</body>
</html>
""")
    return t.substitute(config)


# =========================
# One function to write full site
# =========================
def write_full_scrolly_site(
    out_dir: str,
    interactive_html: str,
    viz_filename: str = "yerevan_continuous_two_sliders.html",
    landing_filename: str = "index.html",
    title: str = "Yerevan scrolly",
    extra_html_files: dict[str, str] | None = None,
    embed_extra_filename: str | None = None,
    focus_mode: str = "remove_zoom_effect",
):
    os.makedirs(out_dir, exist_ok=True)

    # Patch the interactive: UI + center zoom + scrolly hooks
    viz_html = interactive_html
    viz_html = patch_interactive_ui_left_right_vertical_sliders(viz_html)
    viz_html = patch_center_map_button_zoom(viz_html, center_zoom=12.0)
    viz_html = patch_interactive_for_scrolly(viz_html)

    viz_path = os.path.join(out_dir, viz_filename)
    with open(viz_path, "w", encoding="utf-8") as f:
        f.write(viz_html)

    if extra_html_files:
        for fname, html_text in extra_html_files.items():
            Path(os.path.join(out_dir, fname)).write_text(html_text, encoding="utf-8")

    # Gallery placeholders
    g1 = [
        {"src": svg_placeholder_data_uri("Context image 1"), "caption": "Replace with assets/ images"},
        {"src": svg_placeholder_data_uri("Context image 2"), "caption": "Replace with assets/ images"},
        {"src": svg_placeholder_data_uri("Context image 3"), "caption": "Replace with assets/ images"},
    ]
    g2 = [
        {"src": svg_placeholder_data_uri("More image 1"), "caption": "Replace with assets/ images"},
        {"src": svg_placeholder_data_uri("More image 2"), "caption": "Replace with assets/ images"},
    ]

    steps = [
        dict(title="Baseline", heading="Baseline assumptions",
             body="Reference case for transport and amenity strength.",
             hint="Scroll to switch scenarios. The model updates immediately.", t=1.00, a=1.00),
        dict(title="Faster transport", heading="Faster transport",
             body="Lower transport factor reduces time costs and can re-balance the fixed point μ.",
             t=0.70, a=1.00),
        dict(title="Slower transport", heading="Slower transport",
             body="Higher transport factor increases the travel-time penalty and may pull μ inward.",
             t=1.50, a=1.00),
        dict(title="Historic pull", heading="Historic amenities matter more",
             body="Scaling amenity-related terms increases the pull of historic proximity and heritage density.",
             t=1.00, a=1.60),
        dict(title="Weaker amenities", heading="Historic amenities matter less",
             body="Reducing amenity strength dampens historic effects relative to other predictors.",
             t=1.00, a=0.60),
    ]

    explain_blocks = [
        dict(heading="Width", body="The model section uses near-full viewport width with small margins."),
        dict(heading="Cards", body="Scenario cards are fixed-width and compact so the map is large."),
        dict(heading="Menu + bar", body="The top menu is fixed; the progress bar appears only when the map is pinned and sits below the menu."),
        dict(heading="Scrolly", body="Scroll steps send values to the iframe via postMessage."),
    ]

    dashboard_page = "dashboard.html"
    dashboard_button_html = ""
    if embed_extra_filename:
        dashboard_button_html = f"""
        <div style="display:flex; flex-direction:column; gap:12px; margin-top: 12px;">
          <a class="btn btnPrimary" href="{_escape(dashboard_page)}">Open theoretical dashboard</a>
        </div>
        """

    config = {
        "PAGE_TITLE": title,
        "BRAND": title,
        "HERO_KICKER": "Urban model explorer",
        "HERO_TITLE": "Moving centers with transport and amenities",
        "HERO_SUB": "Big map in the model section, compact scenario cards, fixed menu, and a pinned-only progress bar below the menu.",
        "CTA_PRIMARY": "Explore the model",
        "CTA_SECONDARY": "Read the explanation",
        "HERO_IMAGE": svg_placeholder_data_uri("Hero image", 1800, 1200),
        "HERO_IMAGE_CAPTION": "Replace with a real image in assets/ and update the path.",
        "G1_TITLE": "Context",
        "G1_SUB": "Supporting visuals before the model.",
        "GALLERY1_HTML": build_gallery_html(g1, cols=3),
        "MODEL_TITLE": "Interactive model",
        "MODEL_SUB": "Scroll-driven scenarios with a sticky frame.",
        "STEPS_HTML": build_steps_html(steps),
        "VIZ_FILENAME": viz_filename,
        "G2_TITLE": "More visuals",
        "G2_SUB": "More figures or screenshots after the model.",
        "GALLERY2_HTML": build_gallery_html(g2, cols=2),
        "EXPLAIN_TITLE": "Explanation",
        "EXPLAIN_SUB": "Short blocks explaining what is happening.",
        "EXPLAIN_BLOCKS": build_explain_blocks(explain_blocks),
        "EXTRA_EXPLAIN_HTML": "",
        "FOOTER_TEXT": f"""
          <div style="display:flex; flex-direction:column; gap:12px;">
            <div>Static bundle. Serve the folder as a website to share it.</div>
            {dashboard_button_html}
          </div>
        """,
    }

    landing_html = build_landing_html(config, focus_mode=focus_mode)
    landing_html = patch_landing_for_model_focus_zoom(landing_html, focus_mode=focus_mode)

    landing_path = os.path.join(out_dir, landing_filename)
    with open(landing_path, "w", encoding="utf-8") as f:
        f.write(landing_html)

    if embed_extra_filename:
        dash_html = build_dashboard_page_html(
            page_title=f"{title} | Dashboard",
            iframe_src=embed_extra_filename,
            iframe_title="Theoretical modelling dashboard",
            height_px=1150,
        )
        Path(os.path.join(out_dir, dashboard_page)).write_text(dash_html, encoding="utf-8")

    return viz_path, landing_path


# =========================
# USAGE
# =========================
# After your model code, you already have:
#   OUT_DIR = "data/yerevan_interactive"
#   html_filled = the generated interactive HTML string
#
thesis_dashboard_filename = "thesis_sectioned_dashboard.html"
viz_path, landing_path = write_full_scrolly_site(
    out_dir=OUT_DIR,
    interactive_html=html_filled,
    viz_filename="yerevan_continuous_two_sliders.html",
    landing_filename="index.html",
    title="Yerevan scrolly",
    embed_extra_filename=thesis_dashboard_filename,
    focus_mode=MODEL_FOCUS_MODE,
)
print(viz_path, landing_path)


data/yerevan_interactive\yerevan_continuous_two_sliders.html data/yerevan_interactive\index.html


In [21]:
import os
import base64
import hashlib
import requests
from pathlib import Path

OWNER = "Nedric53"
REPO = "Armenia"
BRANCH = "main"

# Local folder that contains index.html and yerevan_continuous_two_sliders.html
LOCAL_SITE_DIR = Path("data/yerevan_interactive").resolve()

# Where to put the site inside the repo
REMOTE_DIR = ""  # recommended for GitHub Pages

# GitHub token (better: set as environment variable GITHUB_TOKEN)
TOKEN = os.environ.get("GITHUB_TOKEN", "aaa")

API = "https://api.github.com"

session = requests.Session()
session.headers.update({
    "Authorization": f"token {TOKEN}",
    "Accept": "application/vnd.github+json",
})

def get_file_sha(path_in_repo: str):
    url = f"{API}/repos/{OWNER}/{REPO}/contents/{path_in_repo}"
    r = session.get(url, params={"ref": BRANCH})
    if r.status_code == 200:
        return r.json().get("sha")
    if r.status_code == 404:
        return None
    raise RuntimeError(f"GitHub API error fetching {path_in_repo}: {r.status_code} {r.text}")

def upload_file(local_path: Path, path_in_repo: str, message: str):
    content_bytes = local_path.read_bytes()
    content_b64 = base64.b64encode(content_bytes).decode("utf-8")

    sha = get_file_sha(path_in_repo)

    url = f"{API}/repos/{OWNER}/{REPO}/contents/{path_in_repo}"
    payload = {
        "message": message,
        "content": content_b64,
        "branch": BRANCH,
    }
    if sha:
        payload["sha"] = sha

    r = session.put(url, json=payload)
    if r.status_code not in (200, 201):
        raise RuntimeError(f"Upload failed for {path_in_repo}: {r.status_code} {r.text}")

def iter_files(root: Path):
    for p in root.rglob("*"):
        if p.is_file():
            yield p

if not LOCAL_SITE_DIR.exists():
    raise FileNotFoundError(f"Local site dir not found: {LOCAL_SITE_DIR}")

for p in iter_files(LOCAL_SITE_DIR):
    rel = p.relative_to(LOCAL_SITE_DIR).as_posix()
    repo_path = rel if not REMOTE_DIR else f"{REMOTE_DIR}/{rel}"
    msg = f"Update site: {repo_path}"
    print("Uploading:", repo_path)
    upload_file(p, repo_path, msg)
print("Done. Your site files are in:", f"{REMOTE_DIR}/")



Uploading: dashboard.html
Uploading: index.html
Uploading: thesis_sectioned_dashboard.html
Uploading: yerevan_continuous_two_sliders.html
Uploading: .ipynb_checkpoints/index-checkpoint.html
Uploading: .ipynb_checkpoints/yerevan_continuous_two_sliders-checkpoint.html
Uploading: assets/context_image_1.png
Uploading: assets/context_image_2.png
Uploading: assets/context_image_3.png
Done. Your site files are in: /
